### MICS IRAQ

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/LFS')
[f for f in os.listdir() if 'sav' in f]

['LFS_2021.sav']

In [7]:
#read in data and meta
df_lfs,meta_lfs=pyreadstat.read_sav('LFS_2021.sav', apply_value_formats=False)

col_names_lfs=meta_lfs.column_names_to_labels
col_vals_lfs=meta_lfs.variable_value_labels

In [81]:
df=df_lfs.copy()

#creating having_disability
'''
gen SSHD=0
replace SSHD = . if VIS_SS==. & HEAR_SS ==. & MOB_SS ==. & COM_SS==. & UB_SS==. & COG_SS==.
replace SSHD = 2 if VIS_SS==2 | HEAR_SS ==2 | MOB_SS ==2 | COM_SS==2 |UB_SS==2 | COG_SS==2
replace SSHD = 3 if VIS_SS==3 | HEAR_SS ==3 | MOB_SS ==3 | COM_SS==3 |UB_SS==3 | COG_SS==3
replace SSHD = 4 if VIS_SS==4 | HEAR_SS ==4 | MOB_SS ==4 | COM_SS==4 |UB_SS==4 | COG_SS==4
replace SSHD = 1 if SSHD ==0
drop if SSHD ==.
label define sshdlb 1 "None" 2 "Mild" 3 "Moderate" 4 "Severe"
label values SSHD sshdlb
'''

having_disability_labels={1.0: 'No, no difficulty',
 2.0: 'Yes, some difficulty',
 3.0: 'Yes, a lot of difficulty',
 4.0: 'Cannot do it at all'}

dis_cols=['dif_sight','dif_hear','dif_mobi','dif_conc','dif_comm','dif_upper_body','dif_care']
df['having_disability']=df[dis_cols].apply(lambda x: x.max(), axis=1)
df['having_disability']=df['having_disability'].map(having_disability_labels)

#recode 5 years age groups
c1=(df['dem_ag1']>=0) & (df['dem_ag1']<=4)
c2=(df['dem_ag1']>=5) & (df['dem_ag1']<=9)
c3=(df['dem_ag1']>=10) & (df['dem_ag1']<=14)
c4=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c5=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c6=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c7=(df['dem_ag1']>=30) & (df['dem_ag1']<=34)
c8=(df['dem_ag1']>=35) & (df['dem_ag1']<=39)
c9=(df['dem_ag1']>=40) & (df['dem_ag1']<=44)
c10=(df['dem_ag1']>=45) & (df['dem_ag1']<=49)
c11=(df['dem_ag1']>=50) & (df['dem_ag1']<=54)
c12=(df['dem_ag1']>=55) & (df['dem_ag1']<=59)
c13=(df['dem_ag1']>=60) & (df['dem_ag1']<=64)
c14=(df['dem_ag1']>=65) & (df['dem_ag1']<=69)
c15=(df['dem_ag1']>=70) & (df['dem_ag1']<=74)
c16=(df['dem_ag1']>=75) & (df['dem_ag1']<=79)
c17=(df['dem_ag1']>=80) & (df['dem_ag1']<=84)
c18=(df['dem_ag1']>=85) & (df['dem_ag1']<=89)
c19=(df['dem_ag1']>=90)

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19]

rslt=['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
'40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84','85-89', '90+']

df['age_5']=np.select(cond,rslt,default=np.nan)
#recode age 18+
df['age_18+']=np.select([df['dem_ag1']>=18, df['dem_ag1']<18],['18+','<18'],default=np.nan )

#recode age: 5-9,10-14,15-19,20-24,25-64,65+
df['age25_64']=np.select([(df['dem_ag1']>=25) & (df['dem_ag1']<=64), 
df['dem_ag1']>=65],['25-64','65+'],default=np.nan)

#recode agegroup: 5-9,10-14,15-19,20-24
cond=[c2,c3,c4,c5]
rslt=['5-9','10-14','15-19','20-24']
df['age_5_1']=np.select(cond,rslt,default=np.nan)

#add labels
vars=['dem_sex','area','dif_sight','dif_hear','dif_mobi','dif_conc','dif_care','dif_comm','dif_upper_body']
for v in vars:
    df[v]=df[v].map(col_vals_lfs[v])

In [95]:
# print('Table 1: generating population_age')
# #column index
# newcol=['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
#        '40-44', '45-49','50-54', '55-59', '60-64', '65-69', '70-74',
#        '75-79', '80-84', '85-89', '90+', '18+', '<18']

# xtab_total=[]
# for col in ['age_5', 'age_18+']:

#        xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df[col],
#        rownames=['Area','Disability Type', 'Sex'],
#        colnames=['agegroup'], 
#        values=df['ilo_wgt'], aggfunc='sum',dropna=False)
#        #drop the column 'na' from the dataframe
#        xtab.drop(['nan'], axis=1, inplace=True)
#        xtab_total.append(xtab)

# #append the dataframes
# T=pd.concat(xtab_total, axis=1) 

# #reindex the columns to rearrange them
# T=T.reindex(newcol, axis='columns')
  
# T.to_excel('Table 1 xtab_all_dis_ByAge.xlsx')
# print('Table 1: population_age generated and saved')

######################################################################################################

print('Table 2: generating DisabType_Age')
#column index
newcol=['5-9', '10-14', '15-19', '20-24', '25-64', '65+']

dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
xtab_total=[]

for dis in dis_cols:
       for col in ['age_5_1', 'age25_64']:
              xtab=pd.crosstab([df['area'],df[dis],df['dem_sex']],df[col],
              rownames=['Area','Disability Type', 'Sex'],
              colnames=['agegroup'], 
              values=df['ilo_wgt'], aggfunc='sum',dropna=False)
              #drop the column 'na' from the dataframe
              xtab.drop(['nan'], axis=1, inplace=True)
              #reindex the columns to rearrange them
              xtab=xtab.reindex(newcol, axis='columns')
              xtab_total.append(xtab)

#append the dataframes
T=pd.concat(xtab_total, axis=0) 

T.to_excel('Table 2 DisabType_Age.xlsx')
print('Table1: DisabType_Age generated and saved')

Table 2: generating DisabType_Age
Table1: DisabType_Age generated and saved


In [121]:
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
newcol=['5-9', '10-14', '15-19', '20-24', '25-64', '65+']

xtab_total=[]
for dis in dis_cols[0:1]:
    for col in ['age_5_1', 'age25_64']:
        xtab=pd.crosstab([df['area'],df[dis],df['dem_sex']],df[col],
        rownames=['Area','Disability Type', 'Sex'],
        colnames=['agegroup'], 
        values=df['ilo_wgt'], aggfunc='sum',dropna=False)
        #drop the column 'na' from the dataframe
        xtab.drop(['nan'], axis=1, inplace=True)
        xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=0)
T=T.reindex(newcol, axis='columns')
#add the name of the disability as an index
# T['disability'] = dis
# T.set_index('disability', append=True, inplace=True)
T.columns
#add the diability name as a new level on the columns
#replicate the disability name as much as the column length
#zip to create tuples and then use them as the new columns
d=[dis]*len(T.columns)
T.columns=tuple(zip(d,T.columns))

In [122]:
T

dif_sight                \
                                                5-9         10-14   
Area  Disability Type          Sex                                  
Rural Cannot do it at all      FEMALE  8.178048e+01           NaN   
                               MALE    1.261803e+02           NaN   
      No, no difficulty        FEMALE  8.009756e+05  8.788451e+05   
                               MALE    8.624786e+05  8.688678e+05   
      Yes, a lot of difficulty FEMALE  1.336480e+03  1.686645e+03   
                               MALE    2.647063e+03  1.731227e+03   
      Yes, some difficulty     FEMALE  1.855834e+03  2.876503e+03   
                               MALE    3.302204e+03  1.607932e+03   
Urban Cannot do it at all      FEMALE           NaN           NaN   
                               MALE    3.497487e+02           NaN   
      No, no difficulty        FEMALE  1.655630e+06  1.723460e+06   
                               MALE    1.700010e+06  1.837621e+06   
      Yes, a lot of difficulty FEMALE  5.247243e+02  1.890941e+03   
                               MALE    1.004839e+03  2.735874e+03   
      Yes, some difficulty     FEMALE  1.713594e+04  8.213233e+03   
                               MALE    4.516101e+03  5.740634e+03   
Rural Cannot do it at all      FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      No, no difficulty        FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      Yes, a lot of difficulty FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      Yes, some difficulty     FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
Urban Cannot do it at all      FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      No, no difficulty        FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      Yes, a lot of difficulty FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      Yes, some difficulty     FEMALE           NaN           NaN   
                               MALE             NaN           NaN   

                                                                   \
                                              15-19         20-24   
Area  Disability Type          Sex                                  
Rural Cannot do it at all      FEMALE           NaN  5.536249e+02   
                               MALE             NaN  4.367395e+01   
      No, no difficulty        FEMALE  6.588330e+05  5.524556e+05   
                               MALE    7.661850e+05  6.415608e+05   
      Yes, a lot of difficulty FEMALE  6.014029e+02  3.737465e+02   
                               MALE    1.033545e+03  6.446633e+02   
      Yes, some difficulty     FEMALE  3.541960e+03  2.460644e+03   
                               MALE    5.199016e+03  1.773166e+03   
Urban Cannot do it at all      FEMALE  2.604318e+02  4.900077e+02   
                               MALE             NaN  8.875566e+02   
      No, no difficulty        FEMALE  1.494224e+06  1.380207e+06   
                               MALE    1.535520e+06  1.407221e+06   
      Yes, a lot of difficulty FEMALE  9.807302e+02  3.093701e+03   
                               MALE    3.005626e+03  2.052755e+03   
      Yes, some difficulty     FEMALE  2.458765e+03  3.812325e+03   
                               MALE    4.861988e+03  3.694933e+03   
Rural Cannot do it at all      FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      No, no difficulty        FEMALE           NaN           NaN   
                               MALE             NaN           NaN   
      Yes, a lot of difficulty FEMALE  

In [ ]:
col_names_lfs

In [7]:
df_total=[]

multiindex_cols=[]

for col in ['MT11','internet_use','MT12']:
    #add labels to values
    if col!='internet_use':
        df[col]=df[col].map(col_vals_wm[col])
        
        r=pd.crosstab([df['HH6'],df['disability_combined']],
        [df[col],df['age_ict']],
        rownames=['Area','Disability level'],colnames=[col_names_wm[col],'age'], 
        values=df['wmweight'],aggfunc='sum',dropna=False)
        df_total.append(r)
        #create the multiindex col and append to multiindex_cols
        '''r.columns are MultiIndex([(         'NO', '15-60'),
                        ('NO RESPONSE', '15-60'),
                        (        'YES', '15-60')],
            names=['Own a mobile phone', 'age'])
            we loop over the tuples and insert at index 0 the column name with col_names_wm[col]'''

        for i in r.columns:
            l=list(i)
            l.insert(0,col_names_wm[col])
            multiindex_cols.append(tuple(l))

In [ ]:
xtab=crosstab()
xtab.generate_xtabs()

In [61]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [57]:
df=pd.read_excel('C:/Users/511232/Desktop/PM2.5_CC.xlsx')
cols=['Exposure Upper PM 2.5 ug/m3','Exposure Upper PM 2.5 ug/m3_CC']

df.head()

,Country,ISO3,Year,Exposure Upper PM 2.5 ug/m3,Exposure Upper PM 2.5 ug/m3_CC
0,Algeria,DZA,1990,41.6,2
1,Algeria,DZA,1995,44.3,2
2,Algeria,DZA,2000,49.6,2
3,Algeria,DZA,2005,47.5,2
4,Algeria,DZA,2010,44.1,2


In [71]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

df=pd.read_excel('C:/Users/511232/Desktop/PM2.5_CC.xlsx')
cols=['Exposure Upper PM 2.5 ug/m3','Exposure Upper PM 2.5 ug/m3_CC']


slope=[]
intercept=[]
R2=[]

for col in cols:
    x=np.array(df['Year']).reshape(-1, 1)
    y=np.array(df[col]).reshape(-1, 1)

    regr = LinearRegression()
    regr.fit(x, y)
    intercept.append(regr.intercept_[0])
    slope.append(regr.coef_[0][0])

    pred=regr.predict(x)
    R2.append(r2_score(y, pred))

rslt=pd.DataFrame({'slope':slope, 'intercept':intercept, 'R2':R2})
rslt

,slope,intercept,R2
0,0.094919,-128.447632,0.001202
1,-0.004236,10.019200,0.003371
